Project 1  Rubric.   Boot Camp: Data Visualization and Data Analytics 
Question 4 Analysis
Source Data file: 'clean_states.csv', a subset of ‘USA Real Estate Datatest’ from Kaggle.com https://www.kaggle.com/datasets/ahmedshahriarsakib/usa-real-estate-dataset

Represent the house prices by cities on a plot. Plot in a different color for each city, plot on a map with Latitude and Longtitude of each city either for just NY State. Could also map plot by cities of each State and have the color be different for each State. Scatter Plot each city with different tick / color for each city. 

4 selected states: Connecticut, Massachusetts, New Jersey, New York. 

Average Price and Standard Deviation (std) for each State reveals NY with the highest variance in STD, and Mean being houses with 3 Bedrooms and 2 Bathrooms.  

Assumptions: 1. clean data file may contain duplicates if not removed perhaps on criteria of Sold Date and Address. 


In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import requests
import json
import scipy.stats as st
from scipy.stats import linregress
from pprint import pprint
import os
import geopandas as gpd
import hvplot.pandas


In [2]:
%matplotlib widget

In [3]:
#read csv file
states_df=pd.read_csv("clean_states.csv")
states_df

,Unnamed: 0,price,bed,bath,acre_lot,house_size,state,city
0,27821,579900.0,4.0,3.0,2.34,3320.0,Connecticut,Coventry
1,27826,215000.0,2.0,1.0,0.91,960.0,Connecticut,East Windsor
2,27828,144900.0,2.0,1.0,0.36,860.0,Connecticut,Vernon
3,27829,174900.0,3.0,2.0,0.19,1280.0,Connecticut,Manchester
4,27830,289900.0,4.0,2.0,0.38,1617.0,Connecticut,East Windsor
...,...,...,...,...,...,...,...,...
185883,733133,475000.0,4.0,3.0,0.23,2252.0,New Jersey,South Brunswick
185884,733136,350000.0,2.0,2.0,0.01,1551.0,New Jersey,Piscataway
185885,733146,241000.0,2.0,2.0,0.50,1077.0,New Jersey,North Brunswick
185886,733147,629000.0,5.0,2.0,0.06,2084.0,New Jersey,New Brunswick


In [4]:
#find the Average/ Mean of house prices, and the Standard Deviation. 
states_df.groupby("state").agg(["mean","std"]).round()

C:\Users\Owner\AppData\Local\Temp\ipykernel_24224\3925755002.py:2: FutureWarning: ['city'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  states_df.groupby("state").agg(["mean","std"]).round()


Unnamed: 0               price            bed      bath       \
                    mean       std      mean       std mean  std mean  std   
state                                                                        
Connecticut     366855.0  130123.0  326020.0  146810.0  3.0  1.0  2.0  1.0   
Massachusetts   142859.0   71148.0  645017.0  366150.0  3.0  1.0  2.0  1.0   
New Jersey      646053.0   63552.0  395038.0  202851.0  3.0  1.0  2.0  1.0   
New York        514894.0  104704.0  764468.0  442436.0  3.0  1.0  2.0  1.0   

              acre_lot      house_size         
                  mean  std       mean    std  
state                                          
Connecticut        1.0  1.0     1716.0  655.0  
Massachusetts      0.0  0.0     1860.0  839.0  
New Jersey         0.0  0.0     1760.0  668.0  
New York           0.0  0.0     1798.0  761.0

Std for State of New York has the highest number compared to Connecticut, Massachusetts, and New Jersey
The standard deviation measures how far apart the numbers are from the mean of the data set.
Interestingly, the average number of bedrooms is 3 with 2 bathrooms

In [5]:
states_df.columns

Index(['Unnamed: 0', 'price', 'bed', 'bath', 'acre_lot', 'house_size', 'state',
       'city'],
      dtype='object')

In [6]:
#New York Data in a df ny_cities_df
ny_cities_df = states_df[states_df['state'] == 'New York']
ny_cities_df

,Unnamed: 0,price,bed,bath,acre_lot,house_size,state,city
49777,54258,225000.0,4.0,2.0,0.24,1239.0,New York,Copake
49778,54530,40000.0,2.0,1.0,0.20,690.0,New York,Nassau
49779,54531,125000.0,3.0,2.0,1.00,980.0,New York,Hoosick Falls
49780,54539,299900.0,3.0,1.0,0.32,1608.0,New York,Niverville
49781,54542,379500.0,4.0,3.0,0.28,2080.0,New York,Philmont
...,...,...,...,...,...,...,...,...
63457,732127,1750000.0,2.0,1.0,0.41,1001.0,New York,New York City
63458,732237,1650000.0,1.0,1.0,0.24,718.0,New York,New York City
63459,732292,1591380.0,1.0,2.0,0.05,883.0,New York,New York City
63460,732295,2250000.0,2.0,3.0,0.05,1252.0,New York,New York City


In [7]:
# #drop the column of the number which is the indexing row number from original raw data file /del the ['Unnamed:0'] from ny_cities_df
ny_cities_df = ny_cities_df[['price', 'bed', 'bath', 'acre_lot', 'house_size', 'state', 'city']]
ny_cities_df = ny_cities_df.dropna()
ny_cities_df

,price,bed,bath,acre_lot,house_size,state,city
49777,225000.0,4.0,2.0,0.24,1239.0,New York,Copake
49778,40000.0,2.0,1.0,0.20,690.0,New York,Nassau
49779,125000.0,3.0,2.0,1.00,980.0,New York,Hoosick Falls
49780,299900.0,3.0,1.0,0.32,1608.0,New York,Niverville
49781,379500.0,4.0,3.0,0.28,2080.0,New York,Philmont
...,...,...,...,...,...,...,...
63457,1750000.0,2.0,1.0,0.41,1001.0,New York,New York City
63458,1650000.0,1.0,1.0,0.24,718.0,New York,New York City
63459,1591380.0,1.0,2.0,0.05,883.0,New York,New York City
63460,2250000.0,2.0,3.0,0.05,1252.0,New York,New York City


In [8]:
#rename column 'state' in ny_cities_df to be 'state_name'
ny_cities_df.rename(columns = {"state":"state_name"}, inplace=True)
ny_cities_df.columns

Index(['price', 'bed', 'bath', 'acre_lot', 'house_size', 'state_name', 'city'], dtype='object')

In [9]:
ny_cities_df

,price,bed,bath,acre_lot,house_size,state_name,city
49777,225000.0,4.0,2.0,0.24,1239.0,New York,Copake
49778,40000.0,2.0,1.0,0.20,690.0,New York,Nassau
49779,125000.0,3.0,2.0,1.00,980.0,New York,Hoosick Falls
49780,299900.0,3.0,1.0,0.32,1608.0,New York,Niverville
49781,379500.0,4.0,3.0,0.28,2080.0,New York,Philmont
...,...,...,...,...,...,...,...
63457,1750000.0,2.0,1.0,0.41,1001.0,New York,New York City
63458,1650000.0,1.0,1.0,0.24,718.0,New York,New York City
63459,1591380.0,1.0,2.0,0.05,883.0,New York,New York City
63460,2250000.0,2.0,3.0,0.05,1252.0,New York,New York City


In [10]:
# read source data file for coordinates of latitude and Longitude of US Cities. 
citieslatlng_df = pd.read_csv("uscities.csv")
citieslatlng_df.columns


Index(['city', 'city_ascii', 'state_id', 'state_name', 'county_fips',
       'county_name', 'lat', 'lng', 'population', 'density', 'source',
       'military', 'incorporated', 'timezone', 'ranking', 'zips', 'id'],
      dtype='object')

In [11]:
citieslatlng_df

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36061,New York,40.6943,-73.9249,18713220,10715,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840034016
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1139,-118.4068,12750807,3276,polygon,False,True,America/Los_Angeles,1,90291 90293 90292 91316 91311 90037 90031 9000...,1840020491
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8373,-87.6862,8604203,4574,polygon,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7839,-80.2102,6445545,5019,polygon,False,True,America/New_York,1,33129 33125 33126 33127 33128 33149 33144 3314...,1840015149
4,Dallas,Dallas,TX,Texas,48113,Dallas,32.7936,-96.7662,5743938,1526,polygon,False,True,America/Chicago,1,75287 75098 75233 75254 75251 75252 75253 7503...,1840019440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28333,Gross,Gross,NE,Nebraska,31015,Boyd,42.9461,-98.5697,2,6,polygon,False,True,America/Chicago,3,68719,1840011032
28334,Lotsee,Lotsee,OK,Oklahoma,40143,Tulsa,36.1334,-96.2091,2,39,polygon,False,True,America/Chicago,3,74063,1840021674
28335,The Ranch,The Ranch,MN,Minnesota,27087,Mahnomen,47.3198,-95.6952,2,2,polygon,False,True,America/Chicago,3,56557,1840039629
28336,Shamrock,Shamrock,OK,Oklahoma,40037,Creek,35.9113,-96.5772,2,2,polygon,False,True,America/Chicago,3,74068,1840022701


In [12]:
nycitieslatlng_df=citieslatlng_df[(citieslatlng_df['state_name']=='New York')]
nycitieslatlng_df

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36061,New York,40.6943,-73.9249,18713220,10715,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840034016
18,Brooklyn,Brooklyn,NY,New York,36047,Kings,40.6501,-73.9496,2559903,14221,polygon,False,True,America/New_York,1,11212 11213 11216 11233 11238 11209 11214 1122...,1840034030
19,Queens,Queens,NY,New York,36081,Queens,40.7498,-73.7976,2230722,7885,polygon,False,True,America/New_York,1,11361 11362 11363 11364 11354 11355 11356 1135...,1840034002
34,Manhattan,Manhattan,NY,New York,36061,New York,40.7834,-73.9662,1628706,27545,polygon,False,True,America/New_York,1,10026 10027 10030 10037 10039 10001 10011 1001...,1840034000
39,Bronx,Bronx,NY,New York,36005,Bronx,40.8501,-73.8662,1418207,12862,polygon,False,True,America/New_York,1,10453 10457 10460 10458 10467 10468 10451 1045...,1840033999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27635,Byersville,Byersville,NY,New York,36051,Livingston,42.5839,-77.7914,43,32,polygon,False,False,America/New_York,3,14517,1840024065
27687,Saltaire,Saltaire,NY,New York,36103,Suffolk,40.6388,-73.1949,41,68,polygon,False,True,America/New_York,3,11706,1840005135
27896,Thendara,Thendara,NY,New York,36043,Herkimer,43.7010,-74.9971,30,20,polygon,False,False,America/New_York,3,13472,1840033970
27901,Thousand Island Park,Thousand Island Park,NY,New York,36045,Jefferson,44.2888,-76.0260,30,39,polygon,False,False,America/New_York,3,13692,1840023988


In [13]:
#ny_cities_df = ny_cities_df[['price', 'bed', 'bath', 'acre_lot', 'house_size', 'state', 'city']]
nycitieslatlng_df = nycitieslatlng_df[['state_name', 'city', 'lat', 'lng']]
nycitieslatlng_df

,state_name,city,lat,lng
0,New York,New York,40.6943,-73.9249
18,New York,Brooklyn,40.6501,-73.9496
19,New York,Queens,40.7498,-73.7976
34,New York,Manhattan,40.7834,-73.9662
39,New York,Bronx,40.8501,-73.8662
...,...,...,...,...
27635,New York,Byersville,42.5839,-77.7914
27687,New York,Saltaire,40.6388,-73.1949
27896,New York,Thendara,43.7010,-74.9971
27901,New York,Thousand Island Park,44.2888,-76.0260


In [14]:
nycitieslatlng_df.count()

state_name    880
city          880
lat           880
lng           880
dtype: int64

In [15]:
merge_df = pd.DataFrame
merge_df = pd.merge(ny_cities_df, nycitieslatlng_df, on ="city")



In [16]:
#want to test to see that only one state is in count.  State is NY.  State_name_x and state_name_y are the same, count is 1. 
merge_df.nunique()

price            893
bed                7
bath               4
acre_lot         111
house_size      1321
state_name_x       1
city             106
state_name_y       1
lat              106
lng              106
dtype: int64

In [17]:
#drop unneeded columns
merge_df = merge_df[['state_name_x','state_name_y','price','city','lat','lng']]
merge_df.count()

state_name_x    9625
state_name_y    9625
price           9625
city            9625
lat             9625
lng             9625
dtype: int64

In [18]:
merge_df.columns


Index(['state_name_x', 'state_name_y', 'price', 'city', 'lat', 'lng'], dtype='object')

In [19]:

#merge_df=merge_df.groupby("city").mean().round()
#merge_df


In [20]:
# map_plot_1 = merge_df.hvplot.points(
#     "lng",
#     "lat",
#     geo = True,
#     tiles = "OSM"
# )
# map_plot_1

In [21]:
# #Once the dataframe is create with lat and lng we can try this code:
# price_city_plot = merge_df.hvplot.points(
#                     'lng',
#                     'lat',
#                     size='price',
#                     scale = 1,
#                     geo = True,
#                     tiles ='OSM',
#                     frame_width = 800,
#                     frame_height = 600
#                     )
#                     color = 'city'
                    
# # Display the map
# price_city_plot
